# Exercise 1 - Question 1 (Language Models)

In [ ]:
#Requirements: to be installed in the virtual environment

pip install torch
pip install transformers
pip install datasets
pip install numpy

<a name="ngram_lm"></a>
## Task A: Data Exploration


Let's use an unsupervised dataset (raw corpus) to evaluate language models' perplexity. We use Huggingface's `datasets` library to download needed datasets.
 

Here we use the `Penn Treebank` dataset, featuring a million words of 1989 Wall Street Journal material. The rare words in this version are already replaced with `<unk>` token. The numbers are also replaced with a special token. This token replacement helps us to end up with a more reasonable vocabulary size to work with.


In [ ]:
import numpy as np
import datasets
from datasets import load_dataset
import torch

ptb_dataset = load_dataset("ptb_text_only", split="train")

# splitting dataset in train/test (to be later used for language model evaluation)
ptb_dataset = ptb_dataset.train_test_split(test_size=0.2, seed=1)
ptb_train, ptb_test = ptb_dataset['train'], ptb_dataset['test']

#### Let's have a look at a few samples of the training dataset (and also the structure of the dataset)

In [ ]:
print(f"{ptb_train[0]}\n\n{ptb_train[1]}\n\n{ptb_train[2]}")

During generation with a given language model, we often need to have a `<stop>` token in our vocabulary to terminate the generation of a given sentence/paragraph. In this dataset, every sample is a sentence, and the `<stop>` token should be added to the end of every sample (i.e., end of sentence).

#### Create a new train/test dataset starting from `ptb_train` and `ptb_test` that has a `<stop>` at the end of each sentence. (Note: do not change the structure of the datasets objects, and just change the respective sentences as discussed).
Hint: use the `.map()` functionality of the `datasets` package (read more [here](https://huggingface.co/docs/datasets/process#map])).

In [ ]:
def add_stop_token(input_sample: dict):
    '''
    args:
        input_sample: a dict representing a sample of the dataset. (look above for the dict struture)
    output:
        modified_sample: modified dict adding <stop> at the end of each sentence.
    '''
    # YOUR CODE HERE
    
    
    return modified_sample
    
    
ptb_train = ptb_train.map(add_stop_token)
ptb_test = ptb_test.map(add_stop_token)

For the both `ptb_train` and `ptb_test` datasets, filter out every sample that has less than 3 tokens. it will help remove very short sentences that are not very helpful for training/evaluating a langugage model.

Hint: use `.filter()` functionality of the `datasets` package (read more [here](https://huggingface.co/docs/datasets/process#select-and-filter)).

In [ ]:
# YOUR CODE HERE


#### What are the 10 most frequent tokens in this dataset? Can you spot the token used to replace the numbers in this dataset? How are rare tokens replaced in this dataset?

In [ ]:
# YOUR CODE HERE


## Task B: Fixed-Window Neural Language Models <a name='fixed_window_neural_lm'></a>

This language model take as input a constant number of tokens, and then outputs a probability distribution for the next token. In this section, we assume the underlying model is a FeedForward Network (FFN) with a single hidden layer. This model doesn't have the sparsity issue of N-gram language models, but is always limited to a fixed window of tokens.

In this section, we don't include the training of the model but rather we use a pretrained model on the same training dataset. We evaluate the language model over the `ptb_test` dataset, to show the power of neural language models, when compared to N-gram language models.

More importantly, we use PyTorch modules in this section, so that you get more familiar with its capabilities. Throughout this exercise, we use a `window_size=3` for this model.



Let's first create a dataset of all consecutive tokens of length `window_size` from the `ptb_train` dataset. you can read more about PyTorch datasets and how to create a custom dataset  [here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files).

In [ ]:
from torch.utils.data import Dataset, DataLoader

window_size = 3
vocabulary_size = 10000
word_emb_dim = 100
hidden_dim = 100


class FixedWindowDataset(Dataset):
    # read more about custom datasets at https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
    def __init__(self,
                 train_dataset: datasets.arrow_dataset.Dataset,
                 test_dataset: datasets.arrow_dataset.Dataset,
                 window_size: int,
                 vocabulary_size: int
                ):
        self.prepared_train_dataset = self.prepare_fixed_window_lm_dataset(train_dataset, window_size + 1)
        self.prepared_test_dataset = self.prepare_fixed_window_lm_dataset(test_dataset, window_size + 1)
        
        dataset_vocab = self.get_dataset_vocabulary(train_dataset)
        # defining a dictionary that simply maps tokens to their respective index in the embedding matrix
        self.word_to_index = {word: idx for idx,word in enumerate(dataset_vocab)}
        self.index_to_word = {idx: word for idx,word in enumerate(dataset_vocab)}
        
        assert vocabulary_size > len(dataset_vocab) , f"The dataset vocab size is {len(dataset_vocab)}!"

    def __len__(self):
        return len(self.prepared_train_dataset)
    
    def get_encoded_test_samples(self):
        all_token_lists = [sample.split() for sample in self.prepared_test_dataset]
        all_token_ids = [[self.word_to_index.get(word, self.word_to_index["<unk>"])
                          for word in token_list[:-1]]
                         for token_list in all_token_lists
                        ]
        all_next_token_ids = [self.word_to_index.get(token_list[-1], self.word_to_index["<unk>"]) for 
                              token_list in all_token_lists]
        return torch.tensor(all_token_ids), torch.tensor(all_next_token_ids)
        
    def __getitem__(self, idx):
        # here we need to transform the data to the format we expect at the model input
        token_list = self.prepared_train_dataset[idx].split()
        # having a fallback to <unk> token if an unseen word is encoded.
        token_ids = [self.word_to_index.get(word, self.word_to_index["<unk>"]) for word in token_list[:-1]]
        next_token_id = self.word_to_index.get(token_list[-1], self.word_to_index["<unk>"])
        return torch.tensor(token_ids), torch.tensor(next_token_id)
    
    def decode_idx_to_word(self, token_id):
        return [self.index_to_word[id_.item()] for id_ in token_id]
    
    def get_dataset_vocabulary(self, train_dataset: datasets.arrow_dataset.Dataset):
        vocab = sorted(set(" ".join([sample["sentence"] for sample in train_dataset]).split()))
        # we also add a <start> token to include initial tokens in the sentences in the dataset
        vocab += ["<start>"]
        return vocab
    
    @staticmethod
    def prepare_fixed_window_lm_dataset(target_dataset: datasets.arrow_dataset.Dataset,
                                        window_size: int):
        '''
        Please note that for the very first tokens, they will be added like "<start> <start> Token#1".
        args:
            target_dataset: the target dataset where its consecutive tokens of length 'window_size' should be extracted
            window_size: the window size for the language model
        output:
            prepared_dataset: a list of strings each containing 'window_size' tokens.
        '''
        
        prepared_dataset = []
        
        # YOUR CODE HERE

        
        return prepared_dataset
        
        

In [ ]:
fixed_window_dataset = FixedWindowDataset(ptb_train, ptb_test, window_size, vocabulary_size)

# let's create a simple dataloader for this dataset
train_dataloader =  DataLoader(fixed_window_dataset, batch_size=8, shuffle=True)

Now, let's define the underlying PyTorch model for the language model. You can read more about PyTorch models [here](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html).

**Note**: Here in the forward pass, we compute the negative log-likelihood after passing through the FFN layers. Here we use `torch.nn.LogSoftmax`, as it's numerically more stable than doing seperately `softmax` followed by taking its logarithm.

In [ ]:
import torch.optim as optim

class Fixed_window_language_model(torch.nn.Module):
    def __init__(self, emb_dim, hidden_dim, window_size, vocab_size=10000):
        super().__init__()

        self.window_size = window_size
        self.emb_dim = emb_dim
        self.word_embeddings = torch.nn.Embedding(vocab_size, emb_dim) # word embeddings
        self.linear1 = torch.nn.Linear(window_size * emb_dim, hidden_dim) # first linear layer
        self.activation_func = torch.tanh # the activation function
        self.linear2 = torch.nn.Linear(hidden_dim, vocab_size) # second linear layer
        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)
        self.criterion = torch.nn.NLLLoss()
     
    def forward(self, input_ids, labels):
        inputs_embeds = self.word_embeddings(input_ids)
        concat_input_embed = inputs_embeds.reshape(-1, self.emb_dim * self.window_size)
        hidden_state = self.activation_func( self.linear1(concat_input_embed) )
        logits = self.log_softmax( self.linear2(hidden_state) )
        loss = self.criterion(logits, labels)
        
        return loss

Now let's see how easy it is to train a model with PyTorch! (we provide a trained model in the cell after train, so that you can just start using the model without going through the time-consuming training)

In [ ]:
# defining the model
model_fixed_window = Fixed_window_language_model(emb_dim=word_emb_dim, hidden_dim=hidden_dim,
                                                 window_size=window_size, vocab_size=vocabulary_size)

# defining the optimizer
optimizer = optim.SGD(model_fixed_window.parameters(),
                      lr=0.005,
                      momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a tuple of (context, target)
        context, target = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = model_fixed_window(context, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5000 == 4999. :    # print every 5000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5000:.3f}')
            running_loss = 0.0

print('Finished Training')

# saving the trained model
torch.save(model_fixed_window.state_dict(), "fixed_window_model.pt")

We provide a trained model, so that you can start using it right away

In [ ]:
fixed_window_checkpoint_file = "fixed_window_model.pt"
model_fixed_window.load_state_dict(torch.load(fixed_window_checkpoint_file))

In [ ]:
# context and 'target' ids (target is the next word after the context)
test_token_ids, test_target_ids = fixed_window_dataset.get_encoded_test_samples()

We now have the `test_token_ids`, `test_target_ids` tensors for the test dataset. The `test_token_ids` are the context ids and `test_target_ids` are the respective **next token** (a.k.a. target here) for these contexts.
#### Using the trained model, implement a function that can output the loss for the discussed test dataset. How can we generally decide if the model is overfitted to the train dataset or not?

In [ ]:
def generate_test_dataset_loss(model: torch.nn.Module,
                               test_token_ids: torch.Tensor,
                               test_target_ids: torch.Tensor):
    '''
    args:
        model: fixed-window language model
        test_token_ids: the context ids in a single tensor.
        test_target_ids: the target ids (next token after the context) in a single tensor.
    output:
        avg_test_loss: The average loss of model over test dataset.
    '''
    batch_size = 4
    test_loss = []
    
    
    # YOUR CODE HERE
    

        
    return avg_test_loss


test_dataset_loss = generate_test_dataset_loss(model_fixed_window, test_token_ids, test_target_ids)
print(f"Test dataset loss is {test_dataset_loss}")

#### Using the trained fixed-window model, implemention a function that can output entropy for a given sequence.

In [ ]:
def get_seqeuence_entropy_fixed_window_lm(model: torch.nn.Module,
                                              input_sequence: str,
                                              window_size: int,
                                              word_to_idx: dict):
    '''
    Note that e.g., in order to get the first token probability, you need to pass a sequence
    like "<start> <start> <start>" (prefix padding) to the neural model. In a similar fashion, we need to pass
    "<start> <start> TOKEN#1" for getting the probability of the second token.
    args:
        model: fixed-window language model
        input_sequence: the sequence for which we want to calculate the probability
        window_size: the size of window for the language model
        word_to_idx: a mapping from words to the embedding indices (to encode tokens before being
                     passed to model). You can get this dict from 'fixed_window_dataset.word_to_index'
    output:
        sequence_entropy: the entropy for the input sequence using the trained model
    '''
    
    # YOUR CODE HERE

    

    return sequence_entropy

#### Compute the perplexity for the trained fixed-window language model over `ptb_test` dataset using the previous function. 

In [ ]:
perplexity = -1

# YOUR CODE HERE



print(f"The fixed-window model perplexity over test dataset is {perplexity}")

### Task C: RNN-based Language Model <a name='rnn_lm'></a>
To address the need for a neural architecture that can proceed with any length input (as opposed to the fixed-window model that can only process a fixed number of tokens), we implement the Recurrent Neural Network (RNN). The core idea behind is that we can apply the same weight W repeatedly.

An advatange of RNN model compared to fixed-window langauage model is that we can pass a given sentence at once, instead of passing it in many windows of size `window_size`. Moreover, the language model has the ability to look behind further that a fixed number of tokens.

 As we already did a neural model training exercise for the previous neural model, we only provide a trained LM at this section, so that you can focus only on the analysis part.
 
You can find the dataset structure as well as the RNN architecture in the `rnn_utils.py` file.

In [ ]:
from rnn_utils import RNNDataset, RNN_language_model

vocabulary_size = 10000
word_emb_dim = 200
hidden_dim = 200

rnn_dataset = RNNDataset(ptb_train, ptb_test, vocabulary_size)

# if gpu is available, we puts the model on it 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Here we need a <pad> token for the RNN model, in order to have a batch of sequences with difference sizes 
pad_idx = rnn_dataset.pad_idx # the index for <pad> token
rnn_model = RNN_language_model(vocab_size=vocabulary_size, emb_dim=word_emb_dim, hidden_dim=hidden_dim,
                               pad_idx=pad_idx)
rnn_model.to(device)

load the model weights using the state_dict in `rnn_model.pt` file.

In [ ]:
# YOUR CODE HERE


As the training of an RNN model is time-consuming, we provide a trained language model on this dataset (`rnn_model.pt`), so that you can just analyze the model performance here.
As mentioned above, as RNN can get sequences with varying lengths, the input sequences should be padded with a special token like `<pad>`, so that we can create a batch of sentences. The output of the defined RNN model (see the architecture detail `rnn_utils.py`) is the model's entropy over the input data.

#### First get the encoded test samples of `ptb_test` dataset, and then pass these (already padded) sentences to the RNN model to get the respective entropy values. Compute the perplexity of the model and compare it with previous approaches.
**HINT**: You can use the `get_encoded_test_samples` function of `rnn_dataset` to get encoded test samples.


In [ ]:
test_perplexity = -1

# YOUR CODE HERE



print(f"The model perplexity is {test_perplexity}")

### Task D: MLM Transformer Language Models (Bonus Question: 10 pts) <a name='rnn_lm'></a>

We are here interested in computing the perplexity of MLM Transformer Language Models such as BERT and RoBERTa. Hoewever, the perplexity for MLM models is not well-defined (The difference with GPT models is illstrated [here](https://huggingface.co/docs/transformers/perplexity).

Instructions: First clone the following repository: https://github.com/asahi417/lmppl.
Install the requirements and follow the instructions to compute the pseudo-perplexity [(Wang and Cho, 2019)](https://aclanthology.org/W19-2304.pdf) of 'BERT-base-uncased', 'BERT-large-uncased', 'RoBERTa-base' and 'RoBERTa-large' for the sentences:
'Shelly ate the sliced banana with a fork' and 'The fork ate the sliced banana'.

Which sentence gets the lowest pseudo-perplexity for each of the models? Which is the best model according to this test?
What is the relation of this test to semantic roles?



In [ ]:
# YOUR CODE HERE

# YOUR ANSWERS HERE